# Process Bahama data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display
import re
from time import gmtime, strftime
import sys
import os
import io

## Load data

In [2]:
df = pd.read_csv('data/edited/addresses_Bahamas.csv')
df

,node_id,address,name,countries,country_codes,sourceID,valid_until,note
0,24000001,"ANNEX FREDERICK & SHIRLEY STS, P.O. BOX N-4805...",NaN,Bahamas,BHS,Bahamas Leaks,The Bahamas Leaks data is current through earl...,NaN
1,24000002,"SUITE E-2,UNION COURT BUILDING, P.O. BOX N-818...",NaN,Bahamas,BHS,Bahamas Leaks,The Bahamas Leaks data is current through earl...,NaN
2,24000003,"LYFORD CAY HOUSE, LYFORD CAY, P.O. BOX N-7785,...",NaN,Bahamas,BHS,Bahamas Leaks,The Bahamas Leaks data is current through earl...,NaN
3,24000004,"P.O. BOX N-3708 BAHAMAS FINANCIAL CENTRE, P.O....",NaN,Bahamas,BHS,Bahamas Leaks,The Bahamas Leaks data is current through earl...,NaN
4,24000005,"LYFORD CAY HOUSE, 3RD FLOOR, LYFORD CAY, P.O. ...",NaN,Bahamas,BHS,Bahamas Leaks,The Bahamas Leaks data is current through earl...,NaN
...,...,...,...,...,...,...,...,...
2253,240492525,"J.P.MORGAN TRUST COMPANY (BAHAMAS) LIMITED, NA...",NaN,Bahamas,BHS,Pandora Papers - Trident Trust,Provider data is current through 2016,NaN
2254,240492536,"MONTAGNE STERLINE CENTRE. EAST BAV STREET, NAS...",NaN,Bahamas,BHS,Pandora Papers - Trident Trust,Provider data is current through 2016,NaN
2255,240491733,"DELTEC HOUSE, LYFORD CAY, PO BOX N-3229, NASSA...",NaN,Bahamas,BHS,Pandora Papers - Trident Trust,Provider data is current through 2016,NaN
2256,240491778,"PROVIDENCE HOUSE, HAST WING, EAST HILL STREET,...",NaN,Bahamas,BHS,Pandora Papers - Trident Trust,Provider data is current through 2016,NaN


In [3]:
df['working_address'] = df['address']

## Create initial `adddress_city` column

In [4]:
nassau_city = ['nassau',
               'nasssau',
               'nasau',
               'massau',
               'nassua',
               'nassaau',
               'nassao',
               'nassu',
               'nassan',
               'naussau']
nassau_dict = {x: 'nassau' for x in nassau_city}
nassau_dict

{'nassau': 'nassau',
 'nasssau': 'nassau',
 'nasau': 'nassau',
 'massau': 'nassau',
 'massa': 'nassau',
 'nassua': 'nassau',
 'nassaau': 'nassau',
 'nassao': 'nassau',
 'nassu': 'nassau',
 'nassan': 'nassau',
 'naussau': 'nassau'}

In [5]:
bahamas_cities = ['freeport',
                  'marsh harbour',
                  'marsh habour',
                  'marsh harboue',
                  "governor's harbour",
                  'governors harbour',
                  'windermere island',
                  'harbour island',
                  'elbow cay',
                  'treasure cay',
                  'gregory town',
                  'spanish wells']

In [6]:
df['address_city'] = df['address'].str.extract("(?P<address_city>{})".format('|'.join(nassau_city + bahamas_cities)), re.IGNORECASE, expand=False).str.title()

In [7]:
df.loc[df['address_city'].isin([x.title() for x in nassau_dict.keys()]), 'address_city'] = 'Nassau'

## Initial `address_island` column

In [8]:
bahamas_islands = ['new providence',
                   'new proividence',
                   'paradise island',
                   'island of paradise',
                   'abaco',
                   'eleuthera',
                   'grand bahama island',
                   'grand island',
                   'grand bahama',
                   'south andros']

In [9]:
df['address_island'] = df['address'].str.extract("(?P<address_island>{})".format('|'.join(bahamas_islands)), re.IGNORECASE, expand=False).str.title()

In [10]:
# Use island names to update city 'Nassau'
df.loc[df['address_city'].isna() & df['address_island'].isin(['New Providence',  'Paradise Island', 'Island Of Paradise', 'New Proividence']), 'address_city'] = 'Nassau'

## Initial `address_street` column

In [11]:
nassau_sts = ['saffrey square',
              'east bay street',
              'bay st',
              'east street',
              'bay street',
              'west bay street',
              'cumberland street',
              'deveaux street',
              'taylor drive',
              'bayview drive east',
              'bayview drive',
              'old fort bay',
              'marlborough & queen streets',
              'malborough & queen streets',
              'east hill market street north',
              'elisabeth avenue and shirley street',
              'elizabeth avenue & shirley street',
              'shirley & charlotte streets',
              'shirley & charlotte street',
              'shirley  & charlotte str',
              'shirley and charlotte',
              'dunmore lane',
              'george street']

In [12]:
df['address_street'] = df['address'].str.extract("(?P<address_street>{})".format('|'.join(nassau_sts)), re.IGNORECASE, expand=False).str.title()

In [13]:
df['address_street'].value_counts(dropna=False)

NaN                                    1560
Saffrey Square                          140
West Bay Street                         113
East Bay Street                          99
Bay St                                   98
Shirley & Charlotte Streets              56
Shirley And Charlotte                    36
George Street                            27
Shirley & Charlotte Street               26
Marlborough & Queen Streets              23
Old Fort Bay                             18
Elizabeth Avenue & Shirley Street        14
Cumberland Street                        12
East Street                               8
Malborough & Queen Streets                6
Dunmore Lane                              6
Taylor Drive                              4
Deveaux Street                            3
Elisabeth Avenue And Shirley Street       3
Shirley  & Charlotte Str                  2
Bayview Drive East                        2
East Hill Market Street North             1
Bayview Drive                   

In [14]:
# Use street names to update city 'Nassau'
df.loc[df['address_city'].isna() & df['address_street'].notna(), 'address_city'] = 'Nassau'

## Handle remaining NANs for `address_city`

In [15]:
df.loc[df['address'].str.lower().str.contains('bay corporate ce'),
       ['address_city', 'address_island', 'address_street', 'address_building']
      ] = ['Nassau', 'New Providence', 'West Bay Street', 'Goodmans Bay Corporate Centre']

In [16]:
df.loc[df['address'].str.lower().str.contains('royal bank of canada')
       , ['address_city', 'address_island', 'address_name']
      ] = ['Nassau', 'New Providence', 'Royal Bank Of Canada Trust Company (Bahamas) Limited']

In [17]:
df.loc[df['address'].str.lower().str.contains('morgan trust company'),
       ['address_city', 'address_island', 'address_name', 'address_building', 'address_pobox', 'address_street']
      ] = ['Nassau', 'New Providence', 'JP Morgan Trust Company (Bahamas) Limited', 'Bahamas Financial Centre', 'PO Box N-4899', 'Shirley And Charlotte Streets']

In [18]:
df.loc[df['address'].str.lower().str.contains('bahamas financial cent'),
       ['address_city', 'address_island', 'address_building', 'address_street']
      ] = ['Nassau', 'New Providence', 'Bahamas Financial Centre', 'Shirley And Charlotte Streets']

In [19]:
df.loc[df['address'].str.lower().str.contains('dominion management'),
       ['address_city', 'address_island', 'address_name', 'address_building', 'address_street', 'address_pobox']
      ] = ['Nassau', 'New Providence', 'Dominion Management Services Limited', 'Dominion House', 'Montrose Avenue', 'PO Box N-9932']

In [20]:
df.loc[df['address'].str.lower().str.contains('nebruck house'),
       ['address_city', 'address_island', 'address_building', 'address_neighborhood']
      ] = ['Nassau', 'New Providence', 'Nebruck house', 'Sandyport']

In [21]:
df.loc[df['address'].str.lower().str.contains('kings court'),
       ['address_city', 'address_island', 'address_building', 'address_street']
      ] = ['Nassau', 'New Providence', 'Kings Court', 'Bay Street']

In [22]:
df.loc[df['address'].str.lower().str.contains('mossack fonseca') & df['address'].str.lower().str.contains('saffrey'),
       ['address_city', 'address_island', 'address_name', 'address_street', 'address_pobox', 'working_addres']
      ] = ['Nassau', 'New Providence', 'Mossack Fonseca & Co (Bahamas) Limited', 'Saffrey Square and Bank Lane', 'PO Box N-8188', 'Suite 205']

In [23]:
df.loc[df['address'].str.lower().str.contains('mossack fonseca') & df['address'].str.lower().str.contains('elizabeth'),
       ['address_city', 'address_island', 'address_name', 'address_building', 'address_street', 'working_addres']
      ] = ['Nassau', 'New Providence', 'Mossack Fonseca & Co (Bahamas) Limited', 'Union Court Building', 'Elizabeth Avenue and Shirley Street', 'Suite E-2']

In [24]:
df.loc[df['address'].str.lower().str.contains('mossack fonseca, bahamas'),
       ['address_city', 'address_island', 'address_name']
      ] = ['Nassau', 'New Providence', 'Mossack Fonseca & Co (Bahamas) Limited']

In [25]:
df.loc[df['address'].str.lower().str.contains('union court') & ~(df['address'].str.lower().str.contains('mossack fonseca')),
       ['address_city', 'address_island', 'address_building', 'address_street']
      ] = ['Nassau', 'New Providence', 'Union Court Building', 'Elizabeth Avenue And Shirley Street']

In [26]:
df.loc[df['address'].str.lower().str.contains('bahamas registrar'),
       ['address_city', 'address_island', 'address_name', 'address_street']
      ] = ['Nassau', 'New Providence', 'Bahamas Registrar Generals Dept', 'Shirley Street']

In [27]:
df.loc[df['address'].str.lower().str.contains('10700'),
       ['address_city', 'address_island', 'address_pobox']
      ] = ['Nassau', 'New Providence', 'PO Box N-10700']

In [28]:
df.loc[df['address'].str.lower().str.contains('bandar seri'),
       ['countries', 'country_codes','address_city', 'address_building', 'address_country', 'address_street']
      ] = ['Brunei', 'BRN', 'Bandar Seri Begawan', 'Britannia House', 'Brunei', 'Jalan Cator']

In [29]:
df.loc[df['address'].str.lower().str.contains('bandar seri'),
       ['countries', 'country_codes','address_city', 'address_building', 'address_country', 'address_street']
      ] = ['Brunei', 'BRN', 'Bandar Seri Begawan', 'Britannia House', 'Brunei', 'Jalan Cator']

In [30]:
df.loc[df['address'].str.lower().str.contains('gibraltar'),
       ['countries', 'country_codes','address_city', 'address_building', 'address_country', 'address_street']
      ] = ['Gibraltar', 'GIB', 'Gibraltar', 'Portland House', 'Gibraltarc', 'Glacis Road']

In [30]:
df.loc[df['address'].str.lower().str.contains('via bigini'),
       ['countries', 'country_codes','address_city', 'address_country', 'address_street']
      ] = ['Italy', 'ITA', 'Massa', 'Italy', 'Via Bigini']

In [31]:
df.loc[df['address'].str.lower().str.contains('waters edge'),
       ['address_city', 'address_island', 'address_street']
      ] = ['Freeport', 'Grand Bahama Island', 'Oceanhill Boulevard']

In [32]:
df.loc[df['address'].str.lower().str.contains('lyford cay'),
       ['address_city', 'address_island', 'address_neighborhood']
      ] = ['Nassau', 'New Providence', 'Lyford Cay']

In [33]:
df['address_city'].value_counts(dropna=False)

Nassau                 2113
Freeport                108
NaN                      12
Marsh Harbour            10
Harbour Island            3
Elbow Cay                 2
Gibraltar                 1
Governor'S Harbour        1
Governors Harbour         1
Windermere Island         1
Bandar Seri Begawan       1
Marsh Harboue             1
Gregory Town              1
Treasure Cay              1
Marsh Habour              1
Spanish Wells             1
Name: address_city, dtype: int64

## Use cities to update `address_island` column

In [34]:
df.loc[(df['address_island'].isna()) & (df['address_city']=='Nassau'), 'address_island'] = 'New Providence'

In [35]:
df.loc[(df['address_island'].isna()) & (df['address_city']=='Freeport'), 'address_island'] = 'Grand Bahama'

In [36]:
df.loc[(df['address_island'].isna()) & df['address'].str.lower().str.contains('governor'), 'address_island'] = 'Eleuthera'

In [37]:
df.loc[df['address'].str.lower().str.contains('harbour island'), 'address_island'] = 'Eleuthera'

## Handle NANs for `address_street`

In [53]:
more_sts = ['2 nd terrace west',
            '2nd terrace and collins avenue',
            '2nd terrace west',
            '7th terrace',
            '8th terrace',
            'alice street',
            'armstrong street',
            'bank lane',
            'bank-lane',
            'bat street',
            'bay & christie streets',
            'bay & east sts',
            'bay & victoria ave',
            'bernard road',
            'bethel drive',
            'beun reitiro road',
            'blake road',
            'blue hill road',
            'bosham close',
            'brace ridge road',
            'bradley st',
            'broadsfield road',
            'caves point',
            'chancery court',
            'chancery lane',
            'charlotte st',
            'charlotte street',
            'chennile avenue',
            'church street',
            'clifton bay drive',
            'cloister drive',
            'collin avenue',
            'collins avenue',
            'collins avenue & 2nd terrace',
            'coral harbour road',
            'cumberland hill street',
            'cumberland st',
            'deanery cumberlandhill street',
            'delaporte point',
            'devonshire street',
            'dowdeswell',
            'dowdeswell & deveaux',
            'dowdeswell st',
            'dowdeswell street',
            'dowdswell,street',
            'dundee bay drive',
            'east avenue',
            'east bav street',
            'east hill st',
            'east hill street',
            'east lyford lane',
            'eastern road',
            'elizabeth avenue',
            'explorers way',
            'forg avenue',
            'fowler st east',
            'fowler st. east',
            'frederic street',
            'frederick & shirley sts',
            'frederick st',
            'frederick street',
            'frederik street',
            'george stre et',
            'grosvenor close',
            'hampshire street',
            'harbour bay plaza',
            'harmony hill road',
            'harrow avenue',
            'harrow street',
            'high st',
            'hill view dr',
            'hollyock road',
            'indigo drive',
            'island lane',
            'ivanhoe road',
            "jamere's plaza5",
            'king & georges streets',
            'king and goerge street',
            'kings & georges street',
            'lagoon court',
            'lomumba st',
            'lyford cay drive',
            'mackey street',
            'maderia street & mount royal avenue',
            'malborough & queen place',
            'malborough & queens street',
            'malborough + queens streets',
            'malborough st. & navy lion rd',
            'market street north',
            'marlboro st',
            'marlborough & qeen streets',
            'marlborough & queen',
            'marlborough & queen st',
            'marlborough & wueen streets',
            'marlborough street',
            'marlborought street',
            'marlborougn & queen',
            'marleborough street',
            'meadows boulevard',
            'melvirn rd',
            'millars court',
            'minnie st',
            'montrose avenue',
            'moseley lane',
            'mount view',
            'mt royal avenue',
            'muirfield drive',
            'musket close',
            'nassau street',
            'nassau street north',
            'nelson street',
            'north victoria avenue',
            'okra hill road',
            'one marina drive',
            'palm breeze drive, carmichael road',
            'paradise island drive',
            'parliament st',
            'parliament street',
            'parliamentstreet',
            'peach street',
            'pinedale street',
            'port of call dr',
            'queen street',
            'r. e. cooper blvd',
            'rainbow road',
            'ranger road',
            'rawson court',
            'rawson sq',
            'resolute road & lyfor cay drive',
            'retirement rd',
            'retirement road',
            'robinson road',
            'rodinson road',
            'rosetta & patton sts',
            'saffery square',
            'saffery square bank lane',
            'sandford drive',
            'sandyport drive',
            'seaview drive',
            'second terrace west',
            'shirley & charlotte',
            'shirley & charolette streets',
            'shirley & christie',
            'shirley st & victoria ave',
            'shirley st and victoria ave',
            'shirley st. & victoria ave',
            'shirley street',
            'shorley street',
            'simone drive',
            'skyline drive',
            'skyline lakes',
            'stanley eastern road',
            'taylor dirve',
            'thomas road',
            'thompson blvd',
            'thompson boulevard',
            'thomson blvd',
            'victoria avenue',
            'village road',
            'village road north',
            'virginia & augusta streets',
            'virginia & augusta sts',
            'virginia street',
            'warwick street',
            'west bay & blake',
            'west bay & blake road',
            'west bay road',
            'west hill street',
            'west street north',
            'western road',
            'westridge drive',
            'windsor avenue',
            'windsor field road',
            'winton highway',
            'woods drive',
            'wulff road',
            'yamacraw hill road',
            'york avenue',
            'york street',
            'yorkshire street',
            'coral road',
            'crossbone close',
            'don mackay boulevard',
            'don mackay, boulevard',
            'double road',
            'east mall drive',
            'east mll drive',
            'gunpont boulevard',
            'jolly roger drive',
            'king & princes street',
            'lillian court',
            'milton street',
            "pioneer's way",
            'santa maria avenue',
            'seabreeze & seascape lanes',
            'the mall drive',
            'west mall dr']

In [55]:
df.loc[df['address_street'].isna(), 'address_street'] = df.loc[df['address_street'].isna(), 'address'].str.extract("({})".format('|'.join(more_sts)), re.IGNORECASE, expand=False).str.title()

## Pull out PO Boxes

In [58]:
df[df['address'].str.lower().str.contains('box')]

,node_id,address,name,countries,country_codes,sourceID,valid_until,note,working_address,address_city,address_island,address_street,address_building,address_name,address_pobox,address_neighborhood,working_addres,address_country
0,24000001,"ANNEX FREDERICK & SHIRLEY STS, P.O. BOX N-4805, NASSAU, BAHAMAS",NaN,Bahamas,BHS,Bahamas Leaks,The Bahamas Leaks data is current through early 2016.,NaN,"ANNEX FREDERICK & SHIRLEY STS, P.O. BOX N-4805, NASSAU, BAHAMAS",Nassau,New Providence,Frederick & Shirley Sts,NaN,NaN,NaN,NaN,NaN,NaN
1,24000002,"SUITE E-2,UNION COURT BUILDING, P.O. BOX N-8188, NASSAU, BAHAMAS",NaN,Bahamas,BHS,Bahamas Leaks,The Bahamas Leaks data is current through early 2016.,NaN,"SUITE E-2,UNION COURT BUILDING, P.O. BOX N-8188, NASSAU, BAHAMAS",Nassau,New Providence,Elizabeth Avenue And Shirley Street,Union Court Building,NaN,NaN,NaN,NaN,NaN
2,24000003,"LYFORD CAY HOUSE, LYFORD CAY, P.O. BOX N-7785, NASSAU, BAHAMAS",NaN,Bahamas,BHS,Bahamas Leaks,The Bahamas Leaks data is current through early 2016.,NaN,"LYFORD CAY HOUSE, LYFORD CAY, P.O. BOX N-7785, NASSAU, BAHAMAS",Nassau,New Providence,NaN,NaN,NaN,NaN,Lyford Cay,NaN,NaN
3,24000004,"P.O. BOX N-3708 BAHAMAS FINANCIAL CENTRE, P.O. BOX N-3708 SHIRLEY & CHARLOTTE STS, NASSAU, BAHAMAS",NaN,Bahamas,BHS,Bahamas Leaks,The Bahamas Leaks data is current through early 2016.,NaN,"P.O. BOX N-3708 BAHAMAS FINANCIAL CENTRE, P.O. BOX N-3708 SHIRLEY & CHARLOTTE STS, NASSAU, BAHAMAS",Nassau,New Providence,Shirley And Charlotte Streets,Bahamas Financial Centre,NaN,NaN,NaN,NaN,NaN
4,24000005,"LYFORD CAY HOUSE, 3RD FLOOR, LYFORD CAY, P.O. BOX N-3024, NASSAU, BAHAMAS",NaN,Bahamas,BHS,Bahamas Leaks,The Bahamas Leaks data is current through early 2016.,NaN,"LYFORD CAY HOUSE, 3RD FLOOR, LYFORD CAY, P.O. BOX N-3024, NASSAU, BAHAMAS",Nassau,New Providence,NaN,NaN,NaN,NaN,Lyford Cay,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2225,240491356,"P.O. BOX N- 3944, SUITE 200B, 2ND FLOOR, CENTRE OF COMMERCE, ONE BAY STREET, NASSAU, BAHAMAS, NASSAU, BAHAMAS",NaN,Bahamas,BHS,Pandora Papers - Trident Trust,Provider data is current through 2017,NaN,"P.O. BOX N- 3944, SUITE 200B, 2ND FLOOR, CENTRE OF COMMERCE, ONE BAY STREET, NASSAU, BAHAMAS, NASSAU, BAHAMAS",Nassau,New Providence,Bay St,NaN,NaN,NaN,NaN,NaN,NaN
2227,240491474,"SUITE 200B, 2ND FLOOR, CENTRE OF COMMERCE, ONE BAY STREET, PO BOX N-3944, NASSAU, BAHAMAS",NaN,Bahamas,BHS,Pandora Papers - Trident Trust,Provider data is current through 2016,NaN,"SUITE 200B, 2ND FLOOR, CENTRE OF COMMERCE, ONE BAY STREET, PO BOX N-3944, NASSAU, BAHAMAS",Nassau,New Providence,Bay St,NaN,NaN,NaN,NaN,NaN,NaN
2229,240491518,"RBC TRUST COMPANY (BAHAMAS) LIMITED, BAYSIDE EXECUTIVE PARK BUILDING 3, P.O. BOX NO. 30-24, NASSAU, BAHAMAS",NaN,Bahamas,BHS,Pandora Papers - Trident Trust,Provider data is current through 2016,NaN,"RBC TRUST COMPANY (BAHAMAS) LIMITED, BAYSIDE EXECUTIVE PARK BUILDING 3, P.O. BOX NO. 30-24, NASSAU, BAHAMAS",Nassau,New Providence,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2255,240491733,"DELTEC HOUSE, LYFORD CAY, PO BOX N-3229, NASSAU, BAHAMAS",NaN,Bahamas,BHS,Pandora Papers - Trident Trust,Provider data is current through 2016,NaN,"DELTEC HOUSE, LYFORD CAY, PO BOX N-3229, NASSAU, BAHAMAS",Nassau,New Providence,NaN,NaN,NaN,NaN,Lyford Cay,NaN,NaN


- df['address']: remove special characters
- PO: optional
- BOX: required
- Box is followed by a single block of letters/numbers

## Set the country

In [87]:
df['address_country'] = df['address'].str.extract('(?P<address_country>bahamas)', re.IGNORECASE, expand=False).str.capitalize()

## Remove stuff from `working_address`

In [88]:
df['working_address'] = df['working_address'].str.replace('bahamas', '', case=False)

## Pull out building names

Now that PO Boxes, islands, and country are out, you can use df['working_address'].notna()

In [ ]:
'c/o','attn'

In [ ]:
'building', 'bldg' ,'house', 'manor', 'center', 'centre', 'ctr','plaza','court','square'

In [49]:
df[df['address_street'].isna()]

,node_id,address,name,countries,country_codes,sourceID,valid_until,note,working_address,address_city,address_island,address_street,address_building,address_name,address_pobox,address_neighborhood,working_addres,address_country
2,24000003,"LYFORD CAY HOUSE, LYFORD CAY, P.O. BOX N-7785, NASSAU, BAHAMAS",NaN,Bahamas,BHS,Bahamas Leaks,The Bahamas Leaks data is current through early 2016.,NaN,"LYFORD CAY HOUSE, LYFORD CAY, P.O. BOX N-7785, NASSAU, BAHAMAS",Nassau,New Providence,NaN,NaN,NaN,NaN,Lyford Cay,NaN,NaN
4,24000005,"LYFORD CAY HOUSE, 3RD FLOOR, LYFORD CAY, P.O. BOX N-3024, NASSAU, BAHAMAS",NaN,Bahamas,BHS,Bahamas Leaks,The Bahamas Leaks data is current through early 2016.,NaN,"LYFORD CAY HOUSE, 3RD FLOOR, LYFORD CAY, P.O. BOX N-3024, NASSAU, BAHAMAS",Nassau,New Providence,NaN,NaN,NaN,NaN,Lyford Cay,NaN,NaN
19,24000020,"P.O. BOX N-9100, NASSAU, BAHAMAS",NaN,Bahamas,BHS,Bahamas Leaks,The Bahamas Leaks data is current through early 2016.,NaN,"P.O. BOX N-9100, NASSAU, BAHAMAS",Nassau,New Providence,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,24000024,"P.O. BOX N 529, NASSAU, BAHAMAS",NaN,Bahamas,BHS,Bahamas Leaks,The Bahamas Leaks data is current through early 2016.,NaN,"P.O. BOX N 529, NASSAU, BAHAMAS",Nassau,New Providence,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,24000026,"3RD FLOOR, MONTAQUE STERLING CENTRE, P.O. BOX N-3242, NASSAU, BAHAMAS",NaN,Bahamas,BHS,Bahamas Leaks,The Bahamas Leaks data is current through early 2016.,NaN,"3RD FLOOR, MONTAQUE STERLING CENTRE, P.O. BOX N-3242, NASSAU, BAHAMAS",Nassau,New Providence,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,24000027,"P.O. BOX GT-2503, NASSAU",NaN,Bahamas,BHS,Bahamas Leaks,The Bahamas Leaks data is current through early 2016.,NaN,"P.O. BOX GT-2503, NASSAU",Nassau,New Providence,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,24000031,"P.O. BOX N-4465, NASSAU",NaN,Bahamas,BHS,Bahamas Leaks,The Bahamas Leaks data is current through early 2016.,NaN,"P.O. BOX N-4465, NASSAU",Nassau,New Providence,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31,24000032,"P.O. BOX N-9306, NASSAU, BAHAMAS",NaN,Bahamas,BHS,Bahamas Leaks,The Bahamas Leaks data is current through early 2016.,NaN,"P.O. BOX N-9306, NASSAU, BAHAMAS",Nassau,New Providence,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32,24000033,"P.O. BOX N-1113, NASSAU, BAHAMAS",NaN,Bahamas,BHS,Bahamas Leaks,The Bahamas Leaks data is current through early 2016.,NaN,"P.O. BOX N-1113, NASSAU, BAHAMAS",Nassau,New Providence,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33,24000034,"P.O. BOX N-1000, NASSAU, BAHAMAS",NaN,Bahamas,BHS,Bahamas Leaks,The Bahamas Leaks data is current through early 2016.,NaN,"P.O. BOX N-1000, NASSAU, BAHAMAS",Nassau,New Providence,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Pull out neighborhoods

In [48]:
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 1000)

In [37]:
pd.set_option('display.max_colwidth', 1000)

In [38]:
df.loc[df['address_city'].isna(), 'working_address']

591                                                  BAHAMAS ADDRESS
592                                       BAHAMAS CO FILE NUM 814872
773               Guernsey as trustees of the Archon Trust (Bahamas)
811                                                      MF, BAHAMAS
1064                                                     THE BAHAMAS
1427                            a Bahamas international business co.
1689         Victoria Point Cay; Mangrove Cay; South Andros; Bahamas
1759                           76 Barbican Road; Kingston 6; Bahamas
1916                                                         BAHAMAS
1918                                                    GRAND BAHAMA
2033    COMPANY MERGED WITH SCOTIABANK (BAHAMAS ) LTD # 51177, 44311
2172                                                         BAHAMAS
Name: working_address, dtype: object